In [5]:
import csv

def get_first_column(csv_file_path):
    first_column = []
    
    with open(csv_file_path, 'r', newline='') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if row:  # Check if the row is not empty
                first_column.append(row[0])
    
    return first_column

print(get_first_column('aapl-balance-sheet-quarterly.csv'))
print(get_first_column('aapl-cash-flow-statement-quarterly.csv'))
print(get_first_column('aapl-income-statement-quarterly.csv'))
print(get_first_column('aapl-ratios-quarterly.csv'))

['\ufeffDate', 'Cash & Equivalents', 'Short-TermInvestments', 'Cash & Cash Equivalents', 'Cash Growth', 'Receivables', 'Inventory', 'Other Current Assets', 'Total Current Assets', 'Property, Plant & Equipment', 'Long-Term Investments', 'Goodwill and Intangibles', 'Other Long-Term Assets', 'Total Long-Term Assets', 'Total Assets', 'Accounts Payable', 'Deferred Revenue', 'Current Debt', 'Total Current Liabilities', 'Other Current Liabilities', 'Long-Term Debt', 'Total Long-Term Liabilities', 'Other Long-Term Liabilities', 'Total Liabilities', 'Total Debt', 'Debt Growth', 'Retained Earnings', 'Comprehensive Income', 'Shareholders Equity', 'Net Cash / Debt', 'Net Cash / Debt Growth', 'Net Cash Per Share', 'Working Capital', 'Book Value Per Share']
['\ufeffDate', 'Net Income', 'Depreciation & Amortization', 'Share-Based Compensation', 'Other Operating Activities', 'Operating Cash Flow', 'Operating Cash Flow Growth', 'Capital Expenditures', 'Acquisitions', 'Change in Investments', 'Other Inv

In [1]:
import pandas as pd
import numpy as np

# Step 1: Load and preprocess the daily stock data
df_daily = pd.read_csv('AAPL.csv')

# Convert 'Date' column to datetime format
df_daily['Date'] = pd.to_datetime(df_daily['Date'])

# Sort the DataFrame by 'Date'
df_daily = df_daily.sort_values('Date')

# Step 2: Define a function to preprocess quarterly data files
def preprocess_quarterly_data(file_path):
    df = pd.read_csv(file_path)
    # Set 'Date' as the index and transpose the DataFrame
    df.set_index('Date', inplace=True)
    df = df.T
    # Reset index to turn the date index into a column
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    # Sort the DataFrame by 'Date'
    df = df.sort_values('Date')

    # Function to clean percentage values
    def clean_percentage(x):
        if isinstance(x, str) and x.endswith('%'):
            return float(x.strip('%')) / 100
        return x

    # Apply the cleaning function to all columns except 'Date'
    for col in df.columns.drop('Date'):
        df[col] = df[col].apply(clean_percentage)

    # Convert all columns to numeric, coercing errors to NaN
    numeric_cols = df.columns.drop('Date')
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    return df

# Step 3: Load and preprocess all quarterly data files
df_income = preprocess_quarterly_data('aapl-income-statement-quarterly.csv')
df_balance = preprocess_quarterly_data('aapl-balance-sheet-quarterly.csv')
df_cash_flow = preprocess_quarterly_data('aapl-cash-flow-statement-quarterly.csv')
df_ratios = preprocess_quarterly_data('aapl-ratios-quarterly.csv')

# Step 4: Merge the quarterly DataFrames on 'Date'
df_quarterly = df_income.merge(df_balance, on='Date', how='outer', suffixes=('', '_balance'))
df_quarterly = df_quarterly.merge(df_cash_flow, on='Date', how='outer', suffixes=('', '_cash_flow'))
df_quarterly = df_quarterly.merge(df_ratios, on='Date', how='outer', suffixes=('', '_ratios'))

# Sort the quarterly data by 'Date'
df_quarterly = df_quarterly.sort_values('Date')

# Reorder columns to have income statement data first, followed by balance sheet, cash flow, and ratios
income_cols = df_income.columns.drop('Date')
balance_cols = df_balance.columns.drop('Date')
cash_flow_cols = df_cash_flow.columns.drop('Date')
ratios_cols = df_ratios.columns.drop('Date')

# Create a list of columns in the desired order
cols_order = ['Date'] + list(income_cols) + list(balance_cols) + list(cash_flow_cols) + list(ratios_cols)
cols_order = pd.Index(cols_order).unique().tolist()  # Remove duplicates

# Reindex the quarterly DataFrame with the new column order
df_quarterly = df_quarterly.reindex(columns=cols_order)

# Step 5: Handle missing values intelligently

# Identify different types of columns
absolute_cols = income_cols.union(balance_cols).union(cash_flow_cols)
ratio_cols = ratios_cols

# Fill absolute value columns
df_quarterly[absolute_cols] = df_quarterly[absolute_cols].fillna(method='ffill')  # Forward-fill
df_quarterly[absolute_cols] = df_quarterly[absolute_cols].fillna(method='bfill')  # Backward-fill

# Interpolate numeric columns
df_quarterly[absolute_cols] = df_quarterly[absolute_cols].interpolate(method='linear', limit_direction='both')

# Fill ratio columns
df_quarterly[ratio_cols] = df_quarterly[ratio_cols].fillna(method='ffill')
df_quarterly[ratio_cols] = df_quarterly[ratio_cols].fillna(method='bfill')
df_quarterly[ratio_cols] = df_quarterly[ratio_cols].interpolate(method='linear', limit_direction='both')

# Step 6: Merge the daily data with the quarterly data
df_merged = pd.merge_asof(df_daily, df_quarterly, on='Date', direction='backward')

# Rearrange columns to place daily data columns after 'Date'
daily_cols = df_daily.columns.tolist()
quarterly_cols = df_quarterly.columns.drop('Date').tolist()
final_cols = ['Date'] + daily_cols[1:] + quarterly_cols
df_merged = df_merged.reindex(columns=final_cols)

# Step 7: Fill any remaining missing values in the merged DataFrame
df_merged[absolute_cols] = df_merged[absolute_cols].fillna(method='ffill').fillna(method='bfill')
df_merged[ratio_cols] = df_merged[ratio_cols].fillna(method='ffill').fillna(method='bfill')

# Optionally, compute ratios if necessary (Example: Return on Equity)
# df_merged['Computed ROE'] = df_merged['Net Income'] / df_merged["Total Shareholders' Equity"]

# Step 8: Save the merged DataFrame to a new CSV file
df_merged.to_csv('AAPL_with_all.csv', index=False)

# Step 9: Display the first few rows of the merged DataFrame
print(df_merged.head())

/var/folders/3n/6fnnx0sn1hqcr0hd8fzxy9740000gn/T/ipykernel_45528/2130781929.py:77: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_quarterly[absolute_cols] = df_quarterly[absolute_cols].fillna(method='ffill')  # Forward-fill
/var/folders/3n/6fnnx0sn1hqcr0hd8fzxy9740000gn/T/ipykernel_45528/2130781929.py:78: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_quarterly[absolute_cols] = df_quarterly[absolute_cols].fillna(method='bfill')  # Backward-fill
/var/folders/3n/6fnnx0sn1hqcr0hd8fzxy9740000gn/T/ipykernel_45528/2130781929.py:84: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_quarterly[ratio_cols] = df_quarterly[ratio_cols].fillna(method='ffill')
/var/folders/3n/6fnnx0sn1hqcr0hd8fzxy9740000gn/T/ipykernel_45528/21

        Date     Close     Volume       Revenue  Revenue Growth  \
0 1992-12-24  0.428448   45964800  2.000292e+09          0.0383   
1 1992-12-28  0.432079   70448000  2.000292e+09          0.0383   
2 1992-12-29  0.432986  116278400  2.000292e+09          0.0383   
3 1992-12-30  0.426633  100587200  2.000292e+09          0.0383   
4 1992-12-31  0.433894   92232000  2.000292e+09          0.0383   

   Cost of Revenue  Gross Profit  Selling, General & Admin  \
0     1.189367e+09   810925000.0               409858000.0   
1     1.189367e+09   810925000.0               409858000.0   
2     1.189367e+09   810925000.0               409858000.0   
3     1.189367e+09   810925000.0               409858000.0   
4     1.189367e+09   810925000.0               409858000.0   

   Research & Development  Operating Expenses  ...  Quick Ratio  \
0             160282000.0         570140000.0  ...         0.91   
1             160282000.0         570140000.0  ...         0.91   
2             160282000